# Test Trainer Sagemaker Object Locally

In [15]:
from sagemaker.local import LocalSession
from sagemaker.estimator import Estimator
import os

In [16]:
root_dir = "/" + os.path.join(*os.getcwd().split("/")[:-1])
sagemaker_session = LocalSession()

sagemaker_role = "arn:aws:iam::12345678910:role/test_role"

In [17]:
def load_env_variables(*env_files):
    env_vars = dict()
    get_values = lambda x: [(x.split("=")[0], x.split("=")[1])]
    for env_file in env_files:
        with open(env_file, "r") as file:
            env_vars.update(
                dict(
                    [
                        (key.strip(), value.strip()) 
                        for line in file.readlines()
                        for key, value in get_values(line)
                    ]
                )
            )
    return env_vars

In [18]:
stage = "staging"
environment = load_env_variables(f"../vars.{stage}.env", "../vars.env")

## Test Locally

In [30]:
# settings
local_docker_image = "koombea_blogs_train_koombea_blogs_train_component"
hyperparameters = {
    "min_count":0,
    "size":300,
    "sg":1,
    "window":15,
    "iter":40,
    "sample": 6e-5,
    "hs": 0,
    "negative": 15,
    "ns_exponent": -0.5,
}

In [31]:
# Initialize estimator
estimator = Estimator(
    image_uri = local_docker_image,
    role = sagemaker_role,
    instance_count = 1,
    instance_type = "local",
    hyperparameters = hyperparameters,
    output_path = "file://{}/{}".format(root_dir, "models"),
    sagemaker_session = sagemaker_session,
    base_job_name = "koombea-blogs-train-component",
    tags = [{"name": "blogs-vector-model"},
            {"staging": stage},
            {"db_name": environment.get("MYSQL_DBNAME")}],
    environment = environment
)

In [32]:
estimator.fit({"training": "file://{}/{}".format(root_dir, "data")})

INFO:sagemaker:Creating training-job with name: koombea-blogs-train-component-2023-02-07-03-53-35-095
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-gq5pe:
    command: train
    container_name: pj35idw5j0-algo-1-gq5pe
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: koombea_blogs_train_koombea_blogs_train_component
    networks:
      sagemaker-local:
        aliases:
        - algo-1-gq5pe
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmp9ydedhh_/algo-1-gq5pe/output/data:/opt/ml/output/data
    - /tm

Creating pj35idw5j0-algo-1-gq5pe ... 
Creating pj35idw5j0-algo-1-gq5pe ... done
Attaching to pj35idw5j0-algo-1-gq5pe
pj35idw5j0-algo-1-gq5pe | 2023-02-07 03:53:37,466 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
pj35idw5j0-algo-1-gq5pe | /envs/python38/bin/python -m pip install -r requirements.txt
pj35idw5j0-algo-1-gq5pe | Requirement already satisfied: gensim<4.0,>=3.8.0 in /envs/python38/lib/python3.8/site-packages (from -r requirements.txt (line 1)) (3.8.3)
pj35idw5j0-algo-1-gq5pe | Requirement already satisfied: scikit-learn==0.24.2 in /envs/python38/lib/python3.8/site-packages (from -r requirements.txt (line 2)) (0.24.2)
pj35idw5j0-algo-1-gq5pe | Requirement already satisfied: pandas==1.2.4 in /envs/python38/lib/python3.8/site-packages (from -r requirements.txt (line 3)) (1.2.4)
pj35idw5j0-algo-1-gq5pe | Requirement already satisfied: python-Levenshtein==0.12.2 in /envs/python38/lib/python3.8/site-packages (from -r requirements.txt (line 4)) (

INFO:root:creating /tmp/tmp9ydedhh_/artifacts/output/data
INFO:root:copying /tmp/tmp9ydedhh_/algo-1-gq5pe/output/success -> /tmp/tmp9ydedhh_/artifacts/output
INFO:root:copying /tmp/tmp9ydedhh_/model/bv_model_en_wp_koombea20stg.bv -> /tmp/tmp9ydedhh_/artifacts/model
INFO:root:copying /tmp/tmp9ydedhh_/model/bv_model_es_wp_koombea20stg.bv -> /tmp/tmp9ydedhh_/artifacts/model
INFO:root:copying /tmp/tmp9ydedhh_/model/idx2ids_mapping_en_wp_koombea20stg.json -> /tmp/tmp9ydedhh_/artifacts/model
INFO:root:copying /tmp/tmp9ydedhh_/model/idx2ids_mapping_es_wp_koombea20stg.json -> /tmp/tmp9ydedhh_/artifacts/model
INFO:root:copying /tmp/tmp9ydedhh_/model/blogs_df_wp_koombea20stg.csv -> /tmp/tmp9ydedhh_/artifacts/model
INFO:root:copying /tmp/tmp9ydedhh_/compressed_artifacts/model.tar.gz -> /home/ec2-user/SageMaker/search/koombea_blogs_train/models
INFO:root:copying /tmp/tmp9ydedhh_/compressed_artifacts/output.tar.gz -> /home/ec2-user/SageMaker/search/koombea_blogs_train/models


===== Job Complete =====
